<H2>Feature Extraction

<H3>Libraries

In [1]:
import os
import numpy as np
import pandas as pd

import statistics
from scipy import stats
import statsmodels.api as sm
from biosppy import signals as bio_signals
from scipy.stats import norm, kurtosis, skew
from scipy.signal import find_peaks,peak_widths

<H3>Parameters

In [2]:
classification = 'binary'
#classification = 'selected'

project = 'athena-i'
train_file = 'wind-train-'
new_train_file = 'feature-train-'

size = 150

dataset_path = "%s/%s/dataset/%s/" % (os.getenv('HOME'), project, classification)

<H3>Functions and Methods

Load Dataset

In [3]:
def load_dataset(path, file):
    x_train = pd.read_csv('%s%sx' % (path, file), delimiter=';')
    y_train = pd.read_csv('%s%sy' % (path, file), delimiter=';')
        
    return(x_train, y_train)

In [4]:
def rmssd(list):
    diff_nni = np.diff(list)#successive differences
    return np.sqrt(np.mean(diff_nni ** 2))

#independent function to calculate AVRR   
def avrr(list):
    return sum(list)/len(list)

#independent function to calculate SDRR   
def csdrr(list):
    return statistics.stdev(list)
    
def pNNx(list):
    length_int = len(list)
    diff_nni = np.diff(list)
    nni_50 = sum(np.abs(diff_nni) > 50)
    return 100 * nni_50 / length_int

#independent function to calculate SD1
def SD1(list):
    diff_nn_intervals = np.diff(list)
    return np.sqrt(np.std(diff_nn_intervals, ddof=1) ** 2 * 0.5)

#independent function to calculate SD2
def SD2(list):
    diff_nn_intervals = np.diff(list)
    return np.sqrt(2 * np.std(list, ddof=1) ** 2 - 0.5 * np.std(\
                   diff_nn_intervals, ddof=1) ** 2)
    
#independent function to calculate SD1/SD2
def SD1overSD2(list):
    diff_nn_intervals = np.diff(list)
    sd1 = np.sqrt(np.std(diff_nn_intervals, ddof=1) ** 2 * 0.5)
    sd2 = np.sqrt(2 * np.std(list, ddof=1) ** 2 - 0.5 * np.std(\
                    diff_nn_intervals, ddof=1) ** 2)
    ratio_sd2_sd1 = sd2 / sd1
    return ratio_sd2_sd1
    
    
#independent function to calculate CSI
def CSI(list):
    diff_nn_intervals = np.diff(list)
    sd1 = np.sqrt(np.std(diff_nn_intervals, ddof=1) ** 2 * 0.5)
    sd2 = np.sqrt(2 * np.std(list, ddof=1) ** 2 - 0.5 * np.std(\
                  diff_nn_intervals, ddof=1) ** 2)
    L=4 * sd1
    T=4 * sd2
    return L/T
       
#independent function to calculate CVI
def CVI(list):
    diff_nn_intervals = np.diff(list)
    sd1 = np.sqrt(np.std(diff_nn_intervals, ddof=1) ** 2 * 0.5)
    sd2 = np.sqrt(2 * np.std(list, ddof=1) ** 2 - 0.5 * np.std(\
                  diff_nn_intervals, ddof=1) ** 2)
    L=4 * sd1
    T=4 * sd2
    return np.log10(L * T)
 
#independent function to calculate modified CVI
def modifiedCVI(list):
    diff_nn_intervals = np.diff(list)
    sd1 = np.sqrt(np.std(diff_nn_intervals, ddof=1) ** 2 * 0.5)
    sd2 = np.sqrt(2 * np.std(list, ddof=1) ** 2 - 0.5 * np.std(\
                  diff_nn_intervals, ddof=1) ** 2)
    L=4 * sd1
    T=4 * sd2
    return L ** 2 / T

In [5]:
# Mean Absolute Value (MAV)
def featureMAV(data):
    return np.mean(np.abs(data), axis=0) 

# Root Mean Square (RMS)
def featureRMS(data):
    return np.sqrt(np.mean(data**2, axis=0))

# Log Root Mean Square (RMS)
def featureLRMS(data):
    return np.log(np.sqrt(np.mean(data**2, axis=0)))

# Waveform length (WL) 
def featureWL(data):
    return np.sum(np.abs(np.diff(data, axis=0)),axis=0)/data.shape[0]

# Kurtosis (KUR)
def featureKUR(data):
    #data = norm.rvs(size=1000, random_state=3)
    return kurtosis(data)

# Skewness (SKEW)
def featureSKEW(data):
    return skew(data)

# Variance (VAR)
def featureVAR(data):
    variance = np.var(data)
    return variance

# AR coefficients (AR)
def featureAR(data):
    AR = [0.0, 0.0, 0.0, 0.0]
    e = 0
    method = "mle"
    AR, e = sm.regression.linear_model.yule_walker(data, order=4, method=method)
    return AR.mean()

In [6]:
#EDA
def mn_eda(data):
    return data.mean()

def std_eda(data):
    return data.std()
    
def max_eda(data):
    return data.max()
        
def min_eda(data):
    return data.min()
        
def dr_eda(data):
    max_eda = data.max()
    min_eda = data.min()
    return max_eda/min_eda

def max_peaks(data):
    if (len(data) == 0):
        return 0
    else:
        return np.max(data)

def mean_gsr(data):
    return np.mean(data)
  
def number_of_peaks(data):
    return len(data)

In [7]:
def feature_extraction_ECG(x_train, y_train, size):
    features = pd.DataFrame()
    df_x_train = pd.DataFrame()
    new_y_train = pd.DataFrame()
    
    tmp_features = x_train.drop(columns=['ECG', 'EDA', 'RSP'])
    for i in range(0, len(x_train), size):
        x1 = featureMAV(x_train['ECG'][i:i+size])
        x2 = featureRMS(x_train['ECG'][i:i+size])
        x3 = featureLRMS(x_train['ECG'][i:i+size])
        x4 = featureWL(x_train['ECG'][i:i+size])
        x5 = featureKUR(x_train['ECG'][i:i+size])
        x6 = featureSKEW(x_train['ECG'][i:i+size])
        x7 = featureVAR(x_train['ECG'][i:i+size])
        x8 = SD1overSD2(x_train['ECG'][i:i+size])
        x9 = rmssd(x_train['ECG'][i:i+size])
        x10 = avrr(x_train['ECG'][i:i+size])
        x11 = csdrr(x_train['ECG'][i:i+size])
        x12 = pNNx(x_train['ECG'][i:i+size])
        x13 = SD2(x_train['ECG'][i:i+size])
        x14 = CSI(x_train['ECG'][i:i+size])
        x15 = CVI(x_train['ECG'][i:i+size])
        x16 = modifiedCVI(x_train['ECG'][i:i+size])
        
        df_x_train = df_x_train.append([[x1, x2, x3, x4, x5,
                                         x6, x7, x8, x9, x10,
                                         x11, x11, x12, x13,
                                         x14, x15, x16]])
        
        new_y_train = new_y_train.append(y_train[i+size-1:i+size])
        
        features = features.append(tmp_features[i+size-1:i+size])
    
    df_x_train = df_x_train.reset_index()
    new_y_train = new_y_train.reset_index()
    features = features.reset_index()
    
    df_x_train = df_x_train.drop(columns=['index'])
    new_y_train = new_y_train.drop(columns=['index'])
    features = features.drop(columns=['index'])
    return(df_x_train, new_y_train, features)

In [8]:
def feature_extraction_RSP(x_train, size):
    df_x_train = pd.DataFrame()
    for i in range(0, len(x_train), size):
        x1 = featureMAV(x_train['RSP'][i:i+size])
        x2 = featureRMS(x_train['RSP'][i:i+size])
        x3 = featureLRMS(x_train['RSP'][i:i+size])
        x4 = featureWL(x_train['RSP'][i:i+size])
        x5 = featureKUR(x_train['RSP'][i:i+size])
        x6 = featureSKEW(x_train['RSP'][i:i+size])
        x7 = featureVAR(x_train['RSP'][i:i+size])
        
        df_x_train = df_x_train.append([[x1, x2, x3, x4,
                                         x5, x6, x7]])
    
    df_x_train = df_x_train.reset_index()
    df_x_train = df_x_train.drop(columns=['index'])
    return df_x_train

In [9]:
def feature_extraction_EDA(x_train, size):
    df_x_train = pd.DataFrame()
    for i in range(0, len(x_train), size):
        x1 = mn_eda(x_train['EDA'][i:i+size])
        x2 = std_eda(x_train['EDA'][i:i+size])
        x3 = max_eda(x_train['EDA'][i:i+size])
        x4 = min_eda(x_train['EDA'][i:i+size])
        x5 = max_peaks(x_train['EDA'][i:i+size])
        x6 = mean_gsr(x_train['EDA'][i:i+size])
        
        df_x_train = df_x_train.append([[x1, x2, x3,
                                         x4, x5, x6]])

    df_x_train = df_x_train.reset_index()
    df_x_train = df_x_train.drop(columns=['index'])
    return df_x_train

In [10]:
def export(x_train, y_train, path, file):
    x_train = x_train.reset_index()
    y_train = y_train.reset_index()

    x_train = x_train.drop(columns=['index'])
    y_train = y_train.drop(columns=['index'])

    x_train.to_csv('%s%sx' % (path, file), sep=';',header=True, index=False)
    y_train.to_csv('%s%sy' % (path, file), sep=';',header=True, index=False)

<h3>Call Methods

In [11]:
x_train, y_train = load_dataset(dataset_path, train_file)

In [12]:
ECG_x_train, new_y_train, features = feature_extraction_ECG(x_train, y_train, size)

In [13]:
EDA_x_train = feature_extraction_EDA(x_train, size)

In [14]:
RSP_x_train = feature_extraction_RSP(x_train, size)

In [15]:
new_x_train = pd.DataFrame()
print("dataframe created")
new_x_train = pd.concat([new_x_train, ECG_x_train], axis=1)
print("ecg: ok")
new_x_train = pd.concat([new_x_train, EDA_x_train], axis=1)
print("eda: ok")
new_x_train = pd.concat([new_x_train, RSP_x_train], axis=1)
print("resp: created")
new_x_train = pd.concat([new_x_train, features], axis=1)
print("features: ok")

dataframe created
ecg: ok
eda: ok
resp: created
features: ok


In [16]:
export(new_x_train, new_y_train, dataset_path, new_train_file)